In [12]:
from icecube import icetray, dataclasses, dataio, phys_services
from I3Tray import I3Tray
from icecube.hdfwriter import I3HDFWriter
from icecube.tableio import I3TableWriter
from icecube.hdfwriter import I3HDFTableService
from icecube.weighting import get_weighted_primary
from glob import glob
import tables
import numpy as np
from I3Tray import NaN
import pickle

In [31]:
filepaths = []
for path in ('/data/sim/IceCube/2012/filtered/level2/CORSIKA-in-ice/11499/00000-00999/*',
        '/data/sim/IceCube/2012/filtered/level2/CORSIKA-in-ice/11362/01000-01999/*',
        '/data/sim/IceCube/2012/filtered/level2/CORSIKA-in-ice/11362/00000-00999/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuTau/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuE/low_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/low_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/low_energy/IC86_flasher_p1=0.3_p2=0.0/l2/2/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuTau/high_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuE/high_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/high_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/2/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuE/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuE/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/2/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/low_energy/IC86_flasher_p1=0.3_p2=0.0/l2/3/*',
        '/data/ana/Cscd/StartingEvents/NuGen_new/NuMu/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/3/*',
): 
    filepaths += glob(path)

In [2]:
with open('/home/dfuchsgruber/NuIntClassification/dom_positions.pkl', 'rb') as f:
    dom_positions = pickle.load(f)
dom_positions

{OMKey(4,64,0): {'x': 115.33000183105469,
  'y': -437.1499938964844,
  'z': 1945.3800048828125},
 OMKey(12,64,0): {'x': 288.75,
  'y': -301.79998779296875,
  'z': 1945.4300537109375},
 OMKey(44,64,0): {'x': -135.24000549316406,
  'y': 29.540000915527344,
  'z': 1945.6099853515625},
 OMKey(79,2,0): {'x': 31.25, 'y': -72.93000030517578, 'z': 178.27999877929688},
 OMKey(64,3,0): {'x': 54.2599983215332,
  'y': 292.9700012207031,
  'z': 490.510009765625},
 OMKey(15,25,0): {'x': -290.6600036621094,
  'y': -307.3800048828125,
  'z': 95.6500015258789},
 OMKey(34,5,0): {'x': -200.5500030517578,
  'y': -74.02999877929688,
  'z': 455.3399963378906},
 OMKey(19,6,0): {'x': 210.47000122070312,
  'y': -209.77000427246094,
  'z': 411.8399963378906},
 OMKey(4,7,0): {'x': 114.38999938964844,
  'y': -461.989990234375,
  'z': 396.5400085449219},
 OMKey(38,9,0): {'x': 292.8999938964844,
  'y': 5.199999809265137,
  'z': 355.239990234375},
 OMKey(8,11,0): {'x': -211.35000610351562,
  'y': -404.4800109863281,

In [3]:
paths = {
    interaction_type : glob('/data/ana/LE/dragon/DRAGON_NuMu_disappearance/level5p/{0}/*/*'.format(interaction_type))
    for interaction_type in ('numu', 'nue', 'nutau')
}

In [4]:
path = '/data/ana/LE/dragon/DRAGON_NuMu_disappearance/level5p/{0}/14600/*'.format('numu')
path = glob(path)[0]
datafile = dataio.I3File(path)

In [8]:
path = '/data/sim/IceCube/2012/filtered/level2/CORSIKA-in-ice/11362/00000-00999/Level2_IC86.2012_corsika.011362.000111.i3.bz2'
#path = '/data/ana/Cscd/StartingEvents/NuGen_new/NuTau/medium_energy/IC86_flasher_p1=0.3_p2=0.0/l2/1/l2_00000001.i3.zst'
datafile = dataio.I3File(path)

In [18]:
datafile.rewind()
frame = datafile.pop_physics()

In [19]:
get_weighted_primary(frame, MCPrimary='MCPrimary')
print(frame['MCPrimary'])

[ I3Particle MajorID : 6849073916313389150
             MinorID : 2049
              Zenith : 1.11851
             Azimuth : 2.42927
                   X : -168334
                   Y : 145144
                   Z : 108223
                Time : -813265
              Energy : 125511
               Speed : 0.299792
              Length : 195199
                Type : PPlus
        PDG encoding : 2212
               Shape : Primary
              Status : NotSet
            Location : Anywhere
]


In [10]:
nu = dataclasses.get_most_energetic_neutrino(frame['I3MCTree'])
print(nu)

None


In [26]:
for p in frame['I3MCTree'].get_primaries():
    print(p.pdg_encoding)

-16


In [28]:
nu.energy

32941.24302246454

datafile.rewind()
try_ = True
while try_:
    try:
        for i in range(1):
            frame = datafile.pop_physics()
            x = frame['SRTTWOfflinePulsesDC']
            #track_reco = frame['MPEFit']
            # print(frame['TimeShift'].value)
            #track_reco = frame['MPEFit']
            #time_shift = frame['TimeShift'].value
            track_reco = frame['trueMuon']
            track_reco.shape = dataclasses.I3Particle.ContainedTrack
            track_reco.length = 1000000
            print(frame['IC86_Dunkman_L6_PegLeg_MultiNest8D_Track'])
            #print(frame['MPEFit'])
            track_time = track_reco.time
            hits = x.apply(frame)
            for omkey, pulses in hits:
                pos = dom_positions[omkey]
                pos = dataclasses.I3Position(pos['x'], pos['y'], pos['z'])
                # Calculate the distance
                dist = np.sqrt((pos.x - track_reco.pos.x)**2 + (pos.y - track_reco.pos.y)**2 + (pos.z - track_reco.pos.z)**2)
                track_reco.shape = dataclasses.I3Particle.ContainedTrack
                cherenkov_time_cont = phys_services.I3Calculator.cherenkov_time(track_reco, pos)
                track_reco.shape = dataclasses.I3Particle.InfiniteTrack
                cherenkov_time_inf = phys_services.I3Calculator.cherenkov_time(track_reco, pos)
                print('\t'.join(map(lambda x: str(x).ljust(15), [
                    cherenkov_time_cont + track_time - pulses[0].time,
                    cherenkov_time_inf + track_time - pulses[0].time,
                    dist,
                    pulses[0].charge
                ])))
                
            try_ = False
    except KeyError:
        continue


In [7]:
speed_of_light_in_ice = 0.299792 / 1.33

In [8]:
path

'/data/ana/LE/dragon/DRAGON_NuMu_disappearance/level5p/numu/14600/Level5p_IC86.2013_genie_numu.014600.000610.i3.bz2'

In [75]:
reco_ic86.time, frame['TimeShift'].value

(9834.182836218852, -9850.768128698981)

In [85]:
def dom_get_expected_time(omkey, reco):
    """ Calculates the expected time for a DOM hit assuming a correct track reconstruction.
    
    Parameters:
    -----------
    omkey : Omkey
        The key to identify the optical module.
    reco : icecube.dataclasses.I3Particle
        The track reconstruction (contained track with length).

    Returns:
    --------
    expected_time : float
        The expected time, relative to the time of the interaction, it would take light to reach the DOM
    """
    dom_position = dataclasses.I3Position(dom_positions[omkey]['x'], dom_positions[omkey]['y'], dom_positions[omkey]['z'])
    # Calculate how long photons from the interaction itself would take to arrive at the DOM
    expected_time = (dom_position - reco.pos).magnitude / speed_of_light_in_ice
    # Calculate how long Cherenkov photons would take to arrive at the DOM
    # Assume the track was infinite to also get time at least for DOMs that recorded scattered light
    reco.shape = dataclasses.I3Particle.InfiniteTrack
    cherenkov_time = phys_services.I3Calculator.cherenkov_time(reco, dom_position)
    # If the origin of those Cherenkov photons on the track was *before* the interaction vertex, light must have come from the interaction itself
    cherenkov_position = phys_services.I3Calculator.cherenkov_position(reco, dom_position)
    a = (cherenkov_position - reco.pos).x / reco.dir.x
    assert np.isclose(a, (cherenkov_position - reco.pos).y / reco.dir.y)
    assert np.isclose(a, (cherenkov_position - reco.pos).z / reco.dir.z)
    if a >= 0:
        # The origin of the Cherenkov light on the track is after the interaction vertex
        expected_time = min(expected_time, cherenkov_time)
    return expected_time + reco.time

In [93]:
datafile.rewind()
ctimes = {}
for i in range(1):
    frame = datafile.pop_physics()
    x = frame['SRTTWOfflinePulsesDC']
    hits = x.apply(frame)
    reco_ic86 = frame['IC86_Dunkman_L6_PegLeg_MultiNest8D_Track']
    reco_ic86.shape = dataclasses.I3Particle.InfiniteTrack
    
    for omkey, pulses in hits:
        dom_pos = dom_positions[omkey]
        i3dom_pos = dataclasses.I3Position(dom_pos['x'], dom_pos['y'], dom_pos['z'])
        cktime = (phys_services.I3Calculator.cherenkov_time(reco_ic86, i3dom_pos))
        
        # Check if the cherenkov position is "after" the interaction vertex (in direction of the track)
        ckpos = phys_services.I3Calculator.cherenkov_position(reco_ic86, i3dom_pos)
        a = (ckpos - reco_ic86.pos).x / reco_ic86.dir.x
        assert np.isclose(a, (ckpos - reco_ic86.pos).y / reco_ic86.dir.y)
        assert np.isclose(a, (ckpos - reco_ic86.pos).z / reco_ic86.dir.z)
        if a < 0:
            cktime = np.nan
        dom_distance = (i3dom_pos - reco_ic86.pos).magnitude
        cascade_time = dom_distance / speed_of_light_in_ice
        for pulse in pulses:
            break
        
        ctimes[omkey] = [dom_get_expected_time(omkey, reco_ic86) - pulse.time]
    print(dataclasses.I3Double(frame['I3MCWeightDict']['InteractionType']))
        
ctimes = np.array(ctimes.values())
        
#ctimes = np.array(ctimes)
#float(np.isnan(ctimes[:,0]).sum()) / ctimes.shape[0]

I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)
I3Double(1)


In [91]:
ctimes[:100]

array([[ -60.22428882],
       [-207.73302765],
       [-173.55513241],
       [  -4.92993777],
       [  -7.7966686 ],
       [-277.91090409],
       [  -0.84768198],
       [ -85.23169648],
       [ -56.14594086],
       [  -7.43435848],
       [-109.16869842],
       [ 474.18824067],
       [  -1.10582272],
       [ -79.26055351],
       [   6.00568352],
       [-658.30141878],
       [ -66.30702214]])

In [215]:
ctimes[~np.isnan(ctimes[:, 0])]

array([[  261.21317116, -9232.95249383],
       [  173.01279996, -9311.12528679],
       [  145.90934957, -9336.16428445],
       ...,
       [  160.53503443, -9697.28197595],
       [  208.28662458, -9604.48932929],
       [  258.46176094, -9544.76728975]])

In [177]:
reco_ic86.pos.x

62.20563993810191

In [173]:
np.isnan(ctime)

True

In [45]:
header = frame['I3EventHeader']
print(header)
header.run_id, header.event_id

[ I3EventHeader:
        StartTime: 2009-07-02 23:39:59.999,996,181,7 UTC
         EndTime : 2009-07-02 23:40:00.000,006,228,9 UTC
           RunID : 1
        SubrunID : 0
         EventID : 489
      SubEventID : 0
  SubEventStream : InIceSplit
]


(1, 489)

In [46]:
for i in range(100):
    frame = datafile.pop_physics()
    try:
        frame['trueMuon']
    except:
        continue
    break


In [ ]:
np.log()

In [15]:
print(frame['trueMuon'])

[ I3Particle MajorID : 2917162579409034029
             MinorID : 3298
              Zenith : 1.62604
             Azimuth : 1.12871
                   X : -3.15177
                   Y : -73.1754
                   Z : -250.04
                Time : 9840.83
              Energy : 1.00704
               Speed : 0.299792
              Length : nan
                Type : NuE
        PDG encoding : 12
               Shape : Null
              Status : NotSet
            Location : InIce
]


In [10]:
print(frame['I3MCWeightDict'])

[Crosssection => 4.29259e-11,
EnergyLost => 0,
GENIEWeight => 0.712278,
GeneratorVolume => 9.81748e+07,
GlobalProbabilityScale => 1.33316e-09,
InjectionSurfaceR => 250,
InteractionProbabilityWeight => 4.75664e-11,
InteractionType => 1,
LengthInVolume => 180.715,
MaxAzimuth => 6.28319,
MaxEnergyLog => 0.60206,
MaxZenith => 3.14159,
MinAzimuth => 0,
MinEnergyLog => 0,
MinZenith => 0,
NEvents => 450000,
OneWeight => 121.765,
PowerLawIndex => 1.9,
PrimaryNeutrinoEnergy => 2.69156,
TargetPDGCode => 1.00008e+09,
TotalDetectionLength => 500,
TotalInteractionProbabilityWeight => 9.49583e-10]


In [29]:
print(frame['trueMuon'])
frame['trueMuon'].length

KeyError: "deserialization failed for object at frame key 'trueMuon' \nmake sure the appropriate library (e.g. dataclasses, simclasses, recclasses, etc..) is imported."

In [28]:
'trueMuon' in f

True

In [19]:
print(frame['trueCascade'])
frame['trueCascade'].energy

[ I3Particle MajorID : 11636985984933118165
             MinorID : 2839
              Zenith : 1.72462
             Azimuth : 5.40195
                   X : 68.4004
                   Y : -110.653
                   Z : -438.211
                Time : 9818.28
              Energy : 1.46971
               Speed : 0.299792
              Length : nan
                Type : Pi0
        PDG encoding : 111
               Shape : Cascade
              Status : NotSet
            Location : InIce
]


1.4697064208872177

In [14]:
print(frame['trueNeutrino'])
frame['trueNeutrino'].energy

[ I3Particle MajorID : 11122464779890084869
             MinorID : 24847
              Zenith : 2.84695
             Azimuth : 1.12943
                   X : 119.551
                   Y : -57.9071
                   Z : -370.633
                Time : 9850.77
              Energy : 4.93331
               Speed : 0.299792
              Length : nan
                Type : NuMuBar
        PDG encoding : -14
               Shape : Primary
              Status : NotSet
            Location : InIce
]


4.93331223398588

In [4]:
x = frame['SRTTWOfflinePulsesDC'].apply(frame)
for omkey, pulses in x:
        print(omkey)
        for pulse in pulses:
            print(pulse)

OMKey(80,33,0)
[I3RecoPulse:
             Time : 10228
           Charge : 0.725
            Width : 16
            Flags : FADC 
]
OMKey(80,36,0)
[I3RecoPulse:
             Time : 10296
           Charge : 1.425
            Width : 2
            Flags : LC ATWD FADC 
]
OMKey(80,37,0)
[I3RecoPulse:
             Time : 10152
           Charge : 1.025
            Width : 2
            Flags : LC ATWD FADC 
]
OMKey(80,38,0)
[I3RecoPulse:
             Time : 10132
           Charge : 0.775
            Width : 2
            Flags : LC ATWD FADC 
]
OMKey(80,39,0)
[I3RecoPulse:
             Time : 10115
           Charge : 1.475
            Width : 2
            Flags : LC ATWD FADC 
]
OMKey(80,43,0)
[I3RecoPulse:
             Time : 10179
           Charge : 0.975
            Width : 16
            Flags : FADC 
]
OMKey(80,46,0)
[I3RecoPulse:
             Time : 10386
           Charge : 0.675
            Width : 16
            Flags : FADC 
]
OMKey(83,20,0)
[I3RecoPulse:
             Time :

In [9]:
particle = dataclasses.get_most_energetic_neutrino(frame['I3MCTree'])
print particle.pdg_encoding

-14


In [159]:
datafile = dataio.I3File('/data/ana/LE/dragon/DRAGON_NuMu_disappearance/level5p/numu/14548/Level5p_IC86.2013_genie_numu.014548.000598.i3.bz2')
datafile.rewind()
while True:
    frame = datafile.pop_physics()
    try:
        track_reco = frame['IC86_Dunkman_L6_PegLeg_MultiNest8D_Track']
        print('Pass')
    except: break

Pass
Pass


In [34]:
print frame['trueCascade']

[ I3Particle MajorID : 11122464779890084869
             MinorID : 24851
              Zenith : 1.70788
             Azimuth : 1.03404
                   X : 119.551
                   Y : -57.9071
                   Z : -370.633
                Time : 9850.77
              Energy : 0.68578
               Speed : 0.299792
              Length : nan
                Type : Pi0
        PDG encoding : 111
               Shape : Cascade
              Status : NotSet
            Location : InIce
]


In [152]:
#datafile = dataio.I3File(paths['numu'][0])
datafile = dataio.I3File('/data/ana/LE/dragon/DRAGON_NuMu_disappearance/level5p/numu/14548/Level5p_IC86.2013_genie_numu.014548.000598.i3.bz2')
datafile.rewind()
frame = datafile.pop_physics()
track_reco = frame['IC86_Dunkman_L6_PegLeg_MultiNest8D_Track']
x = frame['InIcePulses']
pulses = x.apply(frame)
for omkey, pulse in pulses:
    pass
frame.keys()
    #print(omkey, pulses[0].charge)


['InIcePulses',
 'OfflineIceTopHLCTankPulses',
 'InIceDSTPulses',
 'I3DST13',
 'TimeShift',
 'PassedAnyFilter',
 'I3MCWeightDict',
 'I3GENIEResultDict',
 'WaveformRange',
 'IceTopDSTPulses',
 'OfflineIceTopSLCVEMPulses',
 'I3SuperDST',
 'QFilterMask',
 'SimTrimmer',
 'FiniteRecoCuts',
 'WIMPrecoTopoSplitSplitCount',
 'IC86_Dunkman_L6_SANTA_DirectPulses',
 'IC86_Dunkman_L6_PegLeg_MultiNest8D_HDCasc',
 'PoleMuonLinefit',
 'trueMuon',
 'CalibratedWaveformRange',
 'DipoleFit_DCParams',
 'SANTA_LineFit_rusage',
 'SANTA_LineFit',
 'FilterMask_NullSplit0',
 'SRTTWOfflinePulsesDC',
 'LineFit_DCParams',
 'IceTopPulses',
 'MPEFitMuEX',
 'SPEFit2MuEX_FSS',
 'IC86_Dunkman_L4_VICH_Charge',
 'IC86_Dunkman_L4_DeepCoreClassic_CoG',
 'SPEFitSingle_DC',
 'trueCascade',
 'TWOfflinePulsesDC',
 'IC86_Dunkman_L6_PegLeg_MultiNest8D_EMCasc',
 'IC86_Dunkman_L3',
 'IC86_Dunkman_L4_SRTTWOfflinePulsesDC_Variables',
 'SANTA_LineFitParams',
 'SPEFitSingleFitParams',
 'LineFit',
 'OfflineIceTopHLCVEMPulses',
 'IC86_

In [153]:
track_reco = frame['IC86_Dunkman_L6_PegLeg_MultiNest8D_Track']
print track_reco
pos = track_reco.pos
pos.x, pos.y, pos.z
track_reco.dir.azimuth, track_reco.dir.zenith

[ I3Particle MajorID : 11188112778715900715
             MinorID : 389378
              Zenith : 1.47873
             Azimuth : 4.12176
                   X : -42.4986
                   Y : -12.1441
                   Z : -464.432
                Time : 9719.03
              Energy : 6.66
               Speed : 0.299792
              Length : 30
                Type : MuMinus
        PDG encoding : 13
               Shape : ContainedTrack
              Status : NotSet
            Location : Anywhere
]


(4.121763700612314, 1.4787331036550944)

In [151]:
def get_frame_length(frame):
    inicepulses = frame['InIcePulses'].apply(frame)
    length = 0
    for omkey, pulses in inicepulses:
        for pulse in pulses:
            length += 1
    #assert(length == len(inicepulses))
    print(length)
    frame['FrameOffsets'] = dataclasses.I3VectorDouble(np.array([length, length+1], dtype=np.float))
    frame['Test'] = dataclasses.I3Double(33)
    return True

In [6]:

tray = I3Tray()
try:
    tray.AddModule('I3Reader',
                 FilenameList = [paths['numu'][0]])
    tray.AddModule(get_frame_length, 'get_frame_length')
    tray.AddModule(I3TableWriter, 'I3TableWriter', keys=['InIcePulses', 'FrameOffsets'], TableService=I3HDFTableService("foo.hdf5"),
                   SubEventStreams=['InIceSplit'],
                   BookEverything=False
                  )
    tray.Execute(10)
finally:
    tray.Finish()


45
47
68
68


In [7]:
print()

()


In [12]:
with tables.open_file("foo.hdf5") as table_file:
    #print(np.array(table_file.root.InIcePulses.cols.charge).shape)
    #print(np.array(table_file.root.NumPulses.cols.value[:100]))
    print(len(table_file.root.FrameOffsets.cols.item))
    #print(table_file.root.FrameOffsets.cols.item[:7])

8
